In [133]:
import pandas as pd
from pathlib import Path
import random
from datetime import datetime
import folium 

In [167]:
def take_sample_use(df_processed, df_all):
    df2 = df_processed[(df_processed['eventsCnt'] >= 1000) & (df_processed['eventsCnt']<5000)]
    users = list(df2.cuebiq_id.unique())
    id = random.choice(users)
    target_date = df2[df2['cuebiq_id'] == id].iloc[0]['date']
    dfu = df_all[df_all['cuebiq_id'] == id]
    dfu_date = dfu[dfu['date'] == target_date]
    dfu_date[['lat', "lng"]] = dfu_date[['lat', "lng"]].apply(pd.to_numeric, errors='coerce')
    dfu_date.dropna(subset=["lat", "lng"])
    m = visualize_trajectory_with_folium(df=dfu_date, lat_col="lat", lon_col="lng", time_col='date')
    return m

In [131]:
def visualize_trajectory_with_folium(df, lat_col, lon_col, time_col):
    
    # Initialize  map based on coordinates means
    m = folium.Map(location=[df[lat_col].mean(), df[lon_col].mean()])
    
    # Add markers
    feature_group = folium.FeatureGroup("Locations")

    for idx, row in df.iterrows():
        try:
            lat, lon, time = row[lat_col], row[lon_col], row[time_col]
            feature_group.add_child(folium.Circle(location=[lat,lon], radius=10, color='crimson',fill=True, popup=time))
        except Exception as e:
            continue

    m.add_child(feature_group)
    
    return m

In [5]:
cuebic_data_dir = Path('/Users/dmatekenya/wbg/cuebic-raw-data/processed/')
input_csv = cuebic_data_dir.joinpath("input.csv")

In [114]:
def preprocess_raw_data(df):
    
    cols = ['cuebiq_id', 'timestamp', 'lat', 'lng']
    df = df[cols]
    df['date'] = df.apply(lambda x: datetime.fromtimestamp(x['timestamp']).date(), axis=1)
    dfg = df.groupby(['cuebiq_id', "date"]).agg({"date": 'count'})
    dfg2 = dfg.rename(columns={'date': 'eventsCnt'})
    dfg3 = dfg2.reset_index(level=['cuebiq_id', 'date'])
    
    return dfg3

In [61]:
df = pd.read_csv(input_csv)
df2 = preprocess_raw_data(df)

/Users/dmatekenya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/dmatekenya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [172]:
map = take_sample_use(dfg3, df)

In [173]:
map

In [174]:
print(dir(map))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_children', '_env', '_get_self_bounds', '_id', '_name', '_parent', '_png_image', '_repr_html_', '_repr_png_', '_template', '_to_png', 'add_child', 'add_children', 'add_to', 'choropleth', 'control_scale', 'crs', 'fit_bounds', 'get_bounds', 'get_name', 'get_root', 'global_switches', 'height', 'keep_in_front', 'left', 'location', 'objects_to_stay_in_front', 'options', 'png_enabled', 'position', 'render', 'save', 'to_dict', 'to_json', 'top', 'width']


In [117]:
df2 = dfg3[(dfg3['eventsCnt'] >= 500) & (dfg3['eventsCnt']<5000)]

In [176]:
d = map.to_dict()

In [99]:
dfg2 = dfg.reset_index(level=0, inplace=True)

IndexError: Too many levels: Index has only 1 level, not 2

In [100]:
dfg.head()

,cuebiq_id,eventsCnt
date,,
2018-05-16,00006e2256666ca5ac25cc5f2feaaafa7c59c646074d5e...,2
2018-04-30,0000c2c12c6c693e92a32f858248ab4b25b1fc0f77900c...,12
2018-05-01,0000c2c12c6c693e92a32f858248ab4b25b1fc0f77900c...,163
2018-05-02,0000c2c12c6c693e92a32f858248ab4b25b1fc0f77900c...,74
2018-05-03,0000c2c12c6c693e92a32f858248ab4b25b1fc0f77900c...,102


In [101]:
dfg.columns

Index(['cuebiq_id', 'eventsCnt'], dtype='object')